In [221]:
import util;
import visualize;
import os;


def parseElaborateLogFile(error_log_file):
    error_data=util.readLinesFromFile(error_log_file);
    loss_string='loss ='
    loss_lines=[(idx,line_curr) for idx,line_curr in enumerate(error_data) if loss_string in line_curr];
    loss_tuple_info=[];
    for idx in range(len(loss_lines)-1):
        start_idx=loss_lines[idx][0];
        end_idx=loss_lines[idx+1][0];
        data_lines=error_data[start_idx+1:end_idx];
        data_lines=[line_curr.strip() for line_curr in data_lines]
        loss_curr=loss_lines[idx+1][1];
        loss_curr=loss_curr[loss_curr.index(loss_string):];
        loss_curr=float(loss_curr[len(loss_string):]);
        loss_tuple_info.append((start_idx,end_idx,loss_curr,data_lines));
    
    return loss_tuple_info;
        
    
    
    



In [222]:
error_log_file='/home/SSD3/maheen-data/horse_project/tps_train_allKP_adam_dummy/log_elaborate.txt'
to_search=\
['/home/SSD3/maheen-data/horse_project/data_check/horse/im/horses_pascal_selected/2009_004662.jpg /home/SSD3/maheen-data/horse_project/data_check/horse/npy/horses_pascal_selected/2009_004662.npy',
'/home/SSD3/maheen-data/horse_project/data_check/horse/im/imagenet_n02374451/n02374451_11539.jpg /home/SSD3/maheen-data/horse_project/data_check/horse/npy/imagenet_n02374451/n02374451_11539.npy',
'/home/SSD3/maheen-data/horse_project/data_check/horse/im/imagenet_n02374451/n02374451_16786.jpg /home/SSD3/maheen-data/horse_project/data_check/horse/npy/imagenet_n02374451/n02374451_16786.npy',
'/home/SSD3/maheen-data/horse_project/data_check/horse/im/imagenet_n02374451/n02374451_4338.jpg /home/SSD3/maheen-data/horse_project/data_check/horse/npy/imagenet_n02374451/n02374451_4338.npy']





to_search=[file_curr[:file_curr.index(' ')] for file_curr in to_search];
to_search=[file_curr.replace('data_check','data_resize') for file_curr in to_search];
loss_tuple_info = parseElaborateLogFile(error_log_file); 

for idx in range(len(loss_tuple_info)):
    data_curr=loss_tuple_info[idx][3];
    for to_search_curr in to_search:
        if to_search_curr in data_curr:
            print 'found!'
            print loss_tuple_info[idx+1][2];

found!
0.326455
found!
0.327254
found!
0.115577
found!
0.713713


In [223]:
def saveImageAndAnno(im_path,npy_path,out_path):
    im=cv2.imread(im_path,1);
    # im=cv2.res tiize(im,(224,224));
    label=np.load(npy_path).astype(np.int);
    x=label[:,0];
    y=label[:,1];
    color=(0,0,255);
    colors=[(0,0,255),(0,255,0),(255,0,0),(255,255,0),(0,255,255)]
    for label_idx in range(len(x)):
        if label[label_idx,2]>0:
            cv2.circle(im,(x[label_idx],y[label_idx]),5,colors[label_idx],-1);
    cv2.imwrite(out_path,im);

In [224]:
import cv2;
import numpy as np;
line_to_find='/home/SSD3/maheen-data/horse_project/data_resize/horse/im/imagenet_n02374451/n02374451_12462.jpg';

input_data_file='/home/SSD3/maheen-data/horse_project/data_resize/horse/matches_5_train_allKP.txt';
input_human_file='/home/SSD3/maheen-data/horse_project/data_resize/aflw/matches_5_train_allKP.txt';

input_data=util.readLinesFromFile(input_data_file);
data_im=[file_curr[:file_curr.index(' ')] for file_curr in input_data];
idx_lines=[idx_line for idx_line,line_curr in enumerate(data_im) if line_curr==line_to_find];
print idx_lines;

input_human=util.readLinesFromFile(input_human_file);
input_human=[file_curr.split(' ') for file_curr in input_human];
input_human_rel=[input_human[idx_curr] for idx_curr in idx_lines];
input_horse_rel=[input_data[idx_curr].split(' ') for idx_curr in idx_lines];
assert len(input_horse_rel)==len(input_human_rel)

out_dir='/home/SSD3/maheen-data/temp/tps_debug_problem';
util.mkdir(out_dir)

combo_arr=input_human_rel+input_horse_rel;
for i in range(len(combo_arr)):
    out_file_curr=os.path.join(out_dir,str(i)+'.jpg');
    saveImageAndAnno(combo_arr[i][0],combo_arr[i][1],out_file_curr);

visualize.writeHTMLForFolder(out_dir);

out_file_horse=os.path.join(out_dir,'out_file_horse.txt')
out_file_human=os.path.join(out_dir,'out_file_human.txt')
util.writeFile(out_file_horse,[' '.join(line_curr) for line_curr in input_horse_rel])
util.writeFile(out_file_human,[' '.join(line_curr) for line_curr in input_human_rel])
print out_file_horse;
print out_file_human;

visualize.writeHTMLForFolder(os.path.join(out_dir,'test_viz'))





[490, 491, 492, 493, 494]
/home/SSD3/maheen-data/temp/tps_debug_problem/out_file_horse.txt
/home/SSD3/maheen-data/temp/tps_debug_problem/out_file_human.txt


In [225]:
def getValidPoints(horse_file,human_file):
    horse_pts=np.load(horse_file);
    human_pts=np.load(human_file);
    check_pts=np.logical_and(horse_pts[:,2]>0,human_pts[:,2]>0);
    valid_horse=horse_pts[check_pts,:];
    valid_human=human_pts[check_pts,:];
    assert valid_horse.shape[0]==valid_human.shape[0];
    assert np.all(valid_horse[:,2]>0);
    assert np.all(valid_human[:,2]>0)

    return valid_horse,valid_human;
    
    

In [226]:
def getPointLineDistanceOld(line_pt_1,line_pt_2,pt):
    assert line_pt_1.size==2;
    assert line_pt_2.size==2;
    assert pt.size==2;
    diffs=line_pt_1-line_pt_2;
    epsilon=1e-8;
    if np.abs(diffs[0])<epsilon:
        distance=np.abs(pt[0]-line_pt_1[0]);
    elif np.abs(diffs[1])<epsilon:
        distance=np.abs(pt[1]-line_pt_1[1]);
    else:
        m=diffs[0]/diffs[1];
        a=-1*m;
        b=1;
        c=m*line_pt_1[1]-line_pt_1[0];
        distance=np.abs(a*pt[1]+b*pt[0]+c);
        distance=distance/np.sqrt(a*a+b*b);
    
    return distance;

In [227]:
def getPointLineDistance(line_pt_1,line_pt_2,pt):
    assert line_pt_1.size==2;
    assert line_pt_2.size==2;
    assert pt.size==2;
    diffs=line_pt_2-line_pt_1;
    epsilon=1e-8;
    if np.sum(np.abs(diffs))<epsilon:
        distance=float('nan')
    elif np.abs(diffs[0])<epsilon:
        distance=np.abs(pt[0]-line_pt_1[0]);
    elif np.abs(diffs[1])<epsilon:
        distance=np.abs(pt[1]-line_pt_1[1]);
    else:
        deno=np.sqrt(np.sum(np.power(diffs,2)));
        numo=(diffs[0]*pt[1])-(diffs[1]*pt[0])+(line_pt_1[0]*line_pt_2[1])-(line_pt_1[1]*line_pt_2[0]);
        numo=np.abs(numo);
        distance=numo/deno;
    
    return distance;

In [228]:
from math import sqrt, fabs
def pdis(a, b, c):
    t = b[0]-a[0], b[1]-a[1]           # Vector ab
    dd = sqrt(t[0]**2+t[1]**2)         # Length of ab
    t = t[0]/dd, t[1]/dd               # unit vector of ab
    n = -t[1], t[0]                    # normal unit vector to ab
    ac = c[0]-a[0], c[1]-a[1]          # vector ac
    return fabs(ac[0]*n[0]+ac[1]*n[1]) # Projection of ac to n (the minimum distance)


In [229]:
def isValidPair((horse_file,human_file,threshold)):
    valid_horse,valid_human=getValidPoints(horse_file,human_file);
    isValid=True;
    if valid_horse.shape[0]<3:
        isValid=True;
    elif (valid_horse.shape[0]==3):
        distances=getAllPointLineDistancesThreePts(valid_human[:,:2]);
        if min(distances)<=threshold:
            isValid=False;
    return isValid;
        

In [230]:
def getAllPointLineDistancesThreePts(pts):
    assert pts.shape[0]==3;
    distances=[];
    for i in range(pts.shape[0]):
        line_pt_1=pts[i,:2];
        line_pt_2=pts[(i+1)%3,:2];
        pt=pts[(i+2)%3,:2];
        distance=getPointLineDistance(line_pt_1,line_pt_2,pt);
        distances.append(distance);
    return distances;


In [231]:
out_dir='/home/SSD3/maheen-data/temp/tps_debug_problem';
out_file_horse=os.path.join(out_dir,'out_file_horse.txt')
out_file_human=os.path.join(out_dir,'out_file_human.txt')
threshold=11.2;
horse_data=util.readLinesFromFile(out_file_horse);
human_data=util.readLinesFromFile(out_file_human);

assert len(horse_data)==len(human_data);
num_files=len(horse_data);

horse_data=[line_curr.split(' ') for line_curr in horse_data];
human_data=[line_curr.split(' ') for line_curr in human_data];


valid_humans=[];
for idx in range(num_files):
    valid_horse,valid_human=getValidPoints(horse_data[idx][1],human_data[idx][1]);
    if valid_human.shape[0]!=3:
        continue;
    valid_humans.append(valid_human);

for valid_human in valid_humans:
    print ('____')
    for i in range(valid_human.shape[0]):
        line_pt_1=valid_human[i,:2];
        line_pt_2=valid_human[(i+1)%3,:2];
        pt=valid_human[(i+2)%3,:2];
        distance=getPointLineDistance(line_pt_1,line_pt_2,pt);
        distance_2=getPointLineDistanceNew(line_pt_1,line_pt_2,pt);
        distance_3=pdis([line_pt_1[1],line_pt_1[0]],[line_pt_2[1],line_pt_2[0]],[pt[1],pt[0]]);
        print distance,distance_2,distance_3;


for idx in range(num_files):
    print isValidPair((horse_data[idx][1],human_data[idx][1],threshold))

    



____
0.230473 0.230473 0.230476183688
2.3938 2.3938 2.39384069641
0.25502 0.25502 0.255017702711
____
14.493 14.493 14.4930517836
70.2129 70.2129 70.2129323472
14.6133 14.6133 14.6132808141
____
5.54196 5.54196 5.54196591232
13.4672 13.4672 13.4672137079
9.25634 9.25634 9.25633523869
____
10.8994 10.8994 10.8993741142
21.6956 21.6956 21.6955547217
20.3328 20.3328 20.3327773952
True
False
True
False
False


In [232]:
import multiprocessing;
input_data_file='/home/SSD3/maheen-data/horse_project/data_resize/horse/matches_5_train_allKP.txt';
input_human_file='/home/SSD3/maheen-data/horse_project/data_resize/aflw/matches_5_train_allKP.txt';
input_human_file_noIm='/home/SSD3/maheen-data/horse_project/data_resize/aflw/matches_5_train_allKP_noIm.txt';

data_horse=util.readLinesFromFile(input_data_file);
input_horse=[file_curr[file_curr.index(' ')+1:] for file_curr in data_horse];
data_human=util.readLinesFromFile(input_human_file);
input_human=[file_curr[file_curr.index(' ')+1:] for file_curr in data_human];
threshold=11.2;

args=[];
for idx,horse_file in enumerate(input_horse):
    human_file=input_human[idx];
    args.append((horse_file,human_file,threshold));

p=multiprocessing.Pool(multiprocessing.cpu_count());
include_list=p.map(isValidPair,args);
include_list=np.array(include_list);
print np.sum(include_list),include_list.size,np.sum(~include_list);

files=[input_data_file,input_human_file,input_human_file_noIm];
for file_curr in files:
    out_file=file_curr[:file_curr.rindex('.')]+'_allBad.txt';
    data_curr=util.readLinesFromFile(file_curr);
    data_rel=np.array(data_curr)[~include_list];
    util.writeFile(out_file,data_rel);
    print out_file;
    

# out_file_horse=input_data_file[:input_data_file.rindex('.')]+'_allBad.txt';
# out_file_human=input_human_file[:input_human_file.rindex('.')]+'_allBad.txt';
# out_file_human_noIm=input_human_file_noIm[:input_human_file_noIm.rindex('.')]+'_allBad.txt';

# util.writeFile(out_file_horse,np.array(data_horse)[~include_list])
# util.writeFile(out_file_human,np.array(data_human)[~include_list])
# util.writeFile(out_file_human_noIm,np.array(data_human_noIm)[~include_list])

    
    

17719 17787 68
/home/SSD3/maheen-data/horse_project/data_resize/horse/matches_5_train_allKP_allBad.txt
/home/SSD3/maheen-data/horse_project/data_resize/aflw/matches_5_train_allKP_allBad.txt
/home/SSD3/maheen-data/horse_project/data_resize/aflw/matches_5_train_allKP_noIm_allBad.txt


In [12]:
def comparativeLossViz(img_dirs,file_post,loss_post,range_batches,range_images,out_file_html,dir_server,img_caption_pre=None):
    img_files_all=[];
    captions_all=[];
    if img_caption_pre is not None:
        assert len(img_caption_pre)==len(img_dirs);
    batch_str=True;
    if range_batches is None:
        range_batches=range(1);
        batch_str=False;
    
    for batch_num in range_batches:
    # range(1,num_batches+1):
        for im_num in range_images:
            for idx_img_dir,img_dir in enumerate(img_dirs):
                if loss_post is None:
                    loss_str='';
                else:
                    loss_all=np.load(os.path.join(img_dir,str(batch_num)+loss_post));
                    if im_num>loss_all.shape[0]:
                        continue;
                    loss_curr=loss_all[im_num-1,0];
                    loss_str="{:10.4f}".format(loss_curr);
                if not batch_str:
                    files_curr=[os.path.join(img_dir,str(im_num)+file_post_curr) for file_post_curr in file_post];
                else:
                    files_curr=[os.path.join(img_dir,str(batch_num)+'_'+str(im_num)+file_post_curr) for file_post_curr in file_post];
                files_curr=[util.getRelPath(file_curr,dir_server) for file_curr in files_curr];
                captions_curr=[os.path.split(file_curr)[1]+' '+loss_str for file_curr in files_curr];
                if img_caption_pre is not None:
                    captions_curr=[img_caption_pre[idx_img_dir]+' '+caption_curr for caption_curr in captions_curr];
                img_files_all.append(files_curr);
                captions_all.append(captions_curr);
    print 'hello again'
    visualize.writeHTML(out_file_html,img_files_all,captions_all,224,224);

In [234]:
dir_server='/home/SSD3/maheen-data';
range_batches=range(1,2);
range_images=range(1,69);
img_dir=['/home/SSD3/maheen-data/temp/tps_debug_problem/test_viz_allBad']
out_file_html=img_dir[0]+'.html';

file_post=['_horse.jpg','_human.jpg','_gtwarp.jpg','_predwarp.jpg']
loss_post='_loss.npy';
comparativeLossViz(img_dir,file_post,loss_post,range_batches,range_images,out_file_html,dir_server)

In [21]:
# '/home/SSD3/maheen-data/training_5_kp_withWarp_fixed_adam__1e-05/test'
# dir_curr='/home/SSD3/maheen-data/training_5_kp_withWarp_fixed_adam__1e-05/test_scrap';
import visualize;
import os;
dir_server='/home/SSD3/maheen-data';
# dir_curr='/home/SSD3/maheen-data/temp/tps_train_allKP_adam_noBad_bn/val_transformed_im'
dir_curr='/home/SSD3/maheen-data/horse_project/ft_horse_allKp_full_halfBack_llr_debug/debug'
img_dirs=[dir_curr];
out_file_html=dir_curr+'.html';
loss_post=None;
range_batches=None;
range_images=range(1,65);
file_post=['.jpg','_org.jpg'];
# visualize.writeHTMLForFolder(dir_curr);
# print dir_curr.replace(dir_server,'http://vision1.idav.ucdavis.edu:1000')+'/'+os.path.split(dir_curr)[1]+'.html';
print 'hello';
comparativeLossViz(img_dirs,file_post,loss_post,range_batches,range_images,out_file_html,dir_server)
print out_file_html.replace(dir_server,'http://vision1.idav.ucdavis.edu:1000')


hello
hello again
http://vision1.idav.ucdavis.edu:1000/horse_project/ft_horse_allKp_full_halfBack_llr_debug/debug.html


In [23]:
import util;
import os;
import visualize;
dir_server='/home/SSD3/maheen-data';
img_dirs=['/home/SSD3/maheen-data/temp/tps_train_allKP_adam_noBad_bn/val_transformed_im',\
          '/home/SSD3/maheen-data/temp/tps_train_allKP_adam_noBad_bn/val_transformed_im_40'];

img_dirs=['/home/SSD3/maheen-data/horse_project/ft_horse_allKp_full_halfBack_llr/test_images']          
file_post=['.jpg','_org.jpg'];
loss_post=None;
range_batches=range(1,3);
range_images=range(1,101);
out_file_html=img_dirs[0]+'.html';

comparativeLossViz(img_dirs,file_post,loss_post,range_batches,range_images,out_file_html,dir_server)
print out_file_html.replace(dir_server,'http://vision1.idav.ucdavis.edu:1000')

hello again
http://vision1.idav.ucdavis.edu:1000/horse_project/ft_horse_allKp_full_halfBack_llr/test_images.html


In [236]:
# path_models='/home/SSD3/maheen-data/training_5_kp_withWarp_fixed_adam__1e-05/intermediate/'
dir_meta='/home/SSD3/maheen-data/training_5_kp_withWarp_fixed_adam__1e-05'
path_to_models=os.path.join(dir_meta,'intermediate')
out_dir_meta=os.path.join(dir_meta,'over_time');
util.mkdir(out_dir_meta);

model_pre=os.path.join(path_to_models,'model_all_');
model_post='.dat'
range_models=range(75,5000,75*8);
print range_models

path_to_th='/home/maheenrashid/Downloads/horses/torch/test_xiuye.th';
out_file_sh=out_dir_meta+'.sh'
commands=[];
for model_idx,model_num in enumerate(range_models):
    model_curr=model_pre+str(model_num)+model_post;
    assert os.path.exists(model_curr);
    out_dir_curr=os.path.join(out_dir_meta,str(model_num));
    command=['th',path_to_th];
    command.extend(['-kp_net_file',model_curr]);
    command.extend(['out_dir',out_dir_curr]);
    command=' '.join(command);
    commands.append(command);
# print commands
util.writeFile(out_file_sh,commands)
print out_file_sh
    



[75, 675, 1275, 1875, 2475, 3075, 3675, 4275, 4875]
/home/SSD3/maheen-data/training_5_kp_withWarp_fixed_adam__1e-05/over_time.sh


In [1]:
import numpy as np
import h5py

h5_file='/home/laoreja/new-deep-landmark/train/vanilla/aflw_40/aflw_vanilla_train_40_0_weight.h5';
with h5py.File(h5_file,'r') as hf:
    print('List of arrays in this file: \n', hf.keys())

('List of arrays in this file: \n', [u'confidence', u'data', u'landmark', u'weight_in', u'weight_out'])
